In [ ]:
from nbdev import *
%nbdev_default_export core

Cells will be exported to RemoteConnect.core,
unless a different module is specified after an export flag: `%nbdev_export special.module`


# module name here

> API details.

In [ ]:
%nbdev_hide
from nbdev.showdoc import *

In [ ]:
%nbdev_export
def say_hello(to):
    print(f"Say hello to {to}")

In [ ]:
say_hello("Rahul")

Say hello to Rahul
